In [14]:
import pandas as pd
import numpy as np
from datetime import timedelta
from pandas_ta.momentum import rsi as ta_rsi
import plotly.graph_objects as go


import os
import pickle
from config import BASE_DIR
print(os.getcwd())
os.chdir(BASE_DIR)
import scripts.functions as fn 




C:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app


In [15]:
klines_file  = './backtest/klines/2d01/Full_BTCUSDT_2d01_2019-01-01_2024-10-15.DataFrame'

with open(klines_file, 'rb') as file:
    df = pickle.load(file)
    #df.set_index('datetime',inplace=True)

#df = fn.ohlc_mirror_v(df)

#Filtrar rango de fechas
#ini = pd.to_datetime('2018-06-26')
#fin = pd.to_datetime('2028-05-01')
#df = df[(df['datetime']>=ini)&(df['datetime']<=fin)]
#df.reset_index(inplace=True)
#df.drop(columns=['index'],inplace=True)


#df.drop(columns=['volume'],inplace=True)

#Quitar parte del dataframe
#df = df[500:1500]
#df.reset_index(inplace=True)
#df.drop(columns=['index'],inplace=True)

In [16]:
df_origin = df.copy()
# Crear una copia para trabajar en Heikin-Ashi con el close promedio entre close anterior y actual
df['HA_close'] = (df['close'].shift(1) + df['close']) / 2
df['HA_open'] = (df['open'].shift(1) + df['close'].shift(1)) / 2
df['HA_high'] = df[['high', 'HA_open', 'HA_close']].max(axis=1)
df['HA_low'] =  df[['low', 'HA_open', 'HA_close']].min(axis=1)

df['close_price'] = df['close']

df['open'] = df['HA_open']
df['close'] = df['HA_close']
df['high'] = df['HA_high']
df['low'] = df['HA_low']
df.drop(columns=['HA_open','HA_close','HA_high','HA_low'],inplace=True)
df.dropna(inplace=True)

df['ha_trend_up'] = np.where(df['low']==df['open'],df['high'],None)
df['ha_trend_down'] = np.where(df['high']==df['open'],df['low'],None)
df['ha_trend_up'].fillna(method='ffill', inplace=True)
df['ha_trend_down'].fillna(method='ffill', inplace=True)

df['sign'] = np.where(df['close']>df['open'],1,-1)
# Crear una columna auxiliar que cambia su valor cada vez que se cambia de signo
df['group'] = (df['sign'] != df['sign'].shift()).cumsum()

# Acumular conteo de velas en cada grupo de alcistas/bajistas
df['streak'] = df.groupby('group').cumcount() + 1

# Opcional: eliminar la columna auxiliar 'group' si no es necesaria
#df.drop(columns='group', inplace=True)

df['buy'] = np.where((df['sign']==1)&(df['streak']>=2),df['close_price'],None)
df['sell'] = np.where((df['sign']==-1)&(df['streak']==1),df['close_price'],None)



df_origin['ha_trend_up'] = df['ha_trend_up']
df_origin['ha_trend_down'] = df['ha_trend_down']
df_origin['buy'] = df['buy']
df_origin['sell'] = df['sell']

#print(df[df['sign']==1]['streak'].describe(),df[df['sign']==-1]['streak'].describe())

df = df_origin.copy()



In [17]:
indicators = [
     {'col': 'ha_trend_up','name': 'ha_trend_up','color': 'green','row': 1, 'mode':'lines',},
     {'col': 'ha_trend_down','name': 'ha_trend_down','color': 'red','row': 1, 'mode':'lines',},
     #{'col': 'st_high','name': 'ST High','color': 'green','row': 1, 'mode':'markers',},
     #   {'col': 'FL_0.0%', 'name': 'FL_0.0%', 'color': 'red', 'row': 1, 'mode': 'lines', }, 
     #   {'col': 'FL_gold1', 'name': 'FL_gold1', 'color': 'orange', 'row': 1, 'mode': 'lines', }, 
     #   {'col': 'FL_gold2', 'name': 'FL_gold2', 'color': 'orange', 'row': 1, 'mode': 'lines', }, 
     #   {'col': 'FL_100.0%', 'name': 'FL_100.0%', 'color': 'green', 'row': 1, 'mode': 'lines', }, 
        ]

indicators_out = [
     #{'col': 'sign_acum','name': 'sign_acum','color': 'yellow','row': 1, 'mode':'lines',},
     #{'col': 'VOL_perc','name': 'Volume_SMA','color': 'green','row': 1, 'mode':'lines',},
        ]

#https://plotly.com/python/reference/scatter/#scatter-marker-symbol
events = [
       #{'df':df,'col':'buy','name': 'BUY','color': 'green','symbol': 'triangle-up' }, 
       #{'df':df,'col':'sell','name': 'SELL','color': 'red','symbol': 'triangle-down' }, 
     ]


fig = fn.ohlc_chart(df,indicators=indicators,indicators_out=indicators_out,events=events,) 
fig.show()